<a href="https://colab.research.google.com/github/PepiMartin28/ETL-Project/blob/main/Extraction_and_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#In this notebook we are going to extract the data from the page "https://fbref.com/".

###We will work with the top five European leagues from the 2000-2001 season onwards.

###First import the necesaries modules

In [9]:
import pandas as pd
import numpy as np
import csv
import time

We are using the next columns:

* league_id
* league_name
* season
* id (this column is created by us)
* name (team name)
* matches_played (matches played in this season)
* wins (matches won in this season)
* draws (matches drawn in this season)
* losses (matches lost in this season)
* season_points (points obtained in this season)
* goals_scored (goals scored in this season)
* shots_on_target (shots made in this season)
* goals_against (goals received in this season)
* shots_against (shots received in this season)
* goal_diff (difference between goals_scored and goals_against)
* clean_sheets (clean sheets made in this season)
* yellow_cards (yellow cards received this season)
* red_cards (red cards received this season)

In [6]:
columns_df = ['league_id', 'league_name', 'season', 'id', 'name', 'matches_played', 'wins', 'draws', 'losses', 'season_points',
               'goals_scored', 'shots_on_target', 'goals_against', 'shots_against', 'goal_diff', 'clean_sheets', 'yellow_cards', 'red_cards']

The next command is the process to extract the data. We use the "time.sleep(10)" function to stop the process for 10 seconds to avoid being blocked from the page for 1 hour.

We use the function of Pandas "read_html" and this function returns a bunch a of tables that we store in the varaible query. We have a condition that when the season is less than 2017 we need the table with index 6 and when the season is 2017 or bigger we need the table with index 8.

In [3]:
leagues = ['Premier-League', 'La-Liga', 'Serie-A', 'Ligue-1', 'Bundesliga']
leagues_id = [9, 12, 11, 13, 20]
id_counter = 0
loaded_teams = {}
rows = []

for idx in range(5):
  for season in range(2000, 2023):
    try:
      query = pd.read_html(f'https://fbref.com/en/comps/{leagues_id[idx]}/{season}-{season+1}/Estadisticas-{season}-{season+1}-{leagues[idx]}')
    except:
      continue

    df_season = query[0]
    df_stats = query[2]
    df_stats.columns = df_stats.columns.droplevel(0)
    df_additional_stats = query[4]
    df_additional_stats.columns = df_additional_stats.columns.droplevel(0)
    df_shots_stats = query[6]
    if season >= 2017:
      df_shots_stats = query[8]
    df_shots_stats.columns = df_shots_stats.columns.droplevel(0)

    teams = [team for team in df_season['Squad'].values]

    for team in teams:
      df_team_season = df_season[df_season['Squad'] == team]
      df_team_stats = df_stats[df_stats['Squad'] == team]
      df_team_additional_stats = df_additional_stats[df_additional_stats['Squad'] == team]
      df_team_shots_stats = df_shots_stats[df_shots_stats['Squad'] == team]

      try:
        team_id = loaded_teams[team]
      except:
        loaded_teams[team] = id_counter
        team_id = id_counter

      rows.append([leagues_id[idx], leagues[idx].replace("-", " "), f'{season}-{season+1}', team_id, team,
                   df_team_season['MP'].values[0], df_team_season['W'].values[0], df_team_season['D'].values[0], df_team_season['L'].values[0],
                   df_team_season['Pts'].values[0], df_team_season['GF'].values[0], df_team_shots_stats['SoT'].values[0],
                   df_team_season['GA'].values[0], df_team_additional_stats['SoTA'].values[0],
                   df_team_season['GF'].values[0] - df_team_season['GA'].values[0],
                   df_team_additional_stats['CS'].values[0], df_team_stats['CrdY'].values[0], df_team_stats['CrdR'].values[0]])

      id_counter += 1

    time.sleep(5)

# with open('put your path here', 'a') as csv_file:
#   writer = csv.writer(csv_file)
#   for row in rows:
#     writer.writerow(row)

In [5]:
len(rows)

2242

In [7]:
teams_df = pd.DataFrame(data=rows, columns=columns_df)

In [8]:
teams_df.head()

,league_id,league_name,season,id,name,matches_played,wins,draws,losses,season_points,goals_scored,shots_on_target,goals_against,shots_against,goal_diff,clean_sheets,yellow_cards,red_cards
0,9,Premier League,2000-2001,0,Manchester Utd,38,24,8,6,80,79,251,31,140,48,17,44.0,3.0
1,9,Premier League,2000-2001,1,Arsenal,38,20,10,8,70,63,267,38,133,25,17,48.0,3.0
2,9,Premier League,2000-2001,2,Liverpool,38,20,9,9,69,71,228,39,165,32,14,53.0,4.0
3,9,Premier League,2000-2001,3,Leeds United,38,20,8,10,68,64,229,43,162,21,11,72.0,3.0
4,9,Premier League,2000-2001,4,Ipswich Town,38,20,6,12,66,57,205,42,157,15,13,32.0,2.0


Now that we have the DataFrame with the raww data, we can start to transform the data.

We want to add the next columns:
* goals_scored/90
* shots_on_target/90
* goals_against/90
* shots_against/90

In [10]:
teams_df['goals_scored/90'] = np.round((teams_df['goals_scored'] / teams_df['matches_played']), 1)

In [11]:
teams_df['goals_against/90'] = np.round((teams_df['goals_against'] / teams_df['matches_played']), 1)

In [12]:
teams_df['shots_on_target/90'] = np.round((teams_df['shots_on_target'] / teams_df['matches_played']), 1)

In [13]:
teams_df['shots_against/90'] = np.round((teams_df['shots_against'] / teams_df['matches_played']), 1)

Now we reorder the columns

In [14]:
teams_df = teams_df[['league_id', 'league_name', 'season', 'id', 'name', 'matches_played', 'wins', 'draws', 'losses', 'season_points',
         'goals_scored', 'goals_scored/90', 'shots_on_target','shots_on_target/90',
         'goals_against', 'goals_against/90', 'shots_against', 'shots_against/90','goal_diff',
         'clean_sheets','yellow_cards', 'red_cards']]

In [15]:
teams_df.head()

,league_id,league_name,season,id,name,matches_played,wins,draws,losses,season_points,...,shots_on_target,shots_on_target/90,goals_against,goals_against/90,shots_against,shots_against/90,goal_diff,clean_sheets,yellow_cards,red_cards
0,9,Premier League,2000-2001,0,Manchester Utd,38,24,8,6,80,...,251,6.6,31,0.8,140,3.7,48,17,44.0,3.0
1,9,Premier League,2000-2001,1,Arsenal,38,20,10,8,70,...,267,7.0,38,1.0,133,3.5,25,17,48.0,3.0
2,9,Premier League,2000-2001,2,Liverpool,38,20,9,9,69,...,228,6.0,39,1.0,165,4.3,32,14,53.0,4.0
3,9,Premier League,2000-2001,3,Leeds United,38,20,8,10,68,...,229,6.0,43,1.1,162,4.3,21,11,72.0,3.0
4,9,Premier League,2000-2001,4,Ipswich Town,38,20,6,12,66,...,205,5.4,42,1.1,157,4.1,15,13,32.0,2.0


Then change the type of the columns "yellow_cards" and "red_cards"

In [16]:
teams_df['yellow_cards'] = np.array(teams_df['yellow_cards'], dtype=int)

In [17]:
teams_df['red_cards'] = np.array(teams_df['red_cards'], dtype=int)

In [18]:
teams_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   league_id           2242 non-null   int64  
 1   league_name         2242 non-null   object 
 2   season              2242 non-null   object 
 3   id                  2242 non-null   int64  
 4   name                2242 non-null   object 
 5   matches_played      2242 non-null   int64  
 6   wins                2242 non-null   int64  
 7   draws               2242 non-null   int64  
 8   losses              2242 non-null   int64  
 9   season_points       2242 non-null   int64  
 10  goals_scored        2242 non-null   int64  
 11  goals_scored/90     2242 non-null   float64
 12  shots_on_target     2242 non-null   int64  
 13  shots_on_target/90  2242 non-null   float64
 14  goals_against       2242 non-null   int64  
 15  goals_against/90    2242 non-null   float64
 16  shots_

Finally we can export the dataframe as a csv file

In [20]:
teams_df.to_csv('put your path here', index=False)